배경 plain 생성, b,g,r 직접지정  (컬러 스포이드 이용)

In [47]:
import cv2
import numpy as np

def make_plain(r,g,b,name):
    save_dir = "C:/Users/Inbody/Desktop/OCR/data/background_settings/plain"
    img = np.full((600, 800, 3), (b, g, r), dtype=np.uint8)
    save_path = save_dir + '/' + name + '.jpg'
    cv2.imshow(save_path ,img)

    cv2.imwrite(save_path, img)

    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [52]:
make_plain(216,142,122,'plain25')

색에 필터 적용하여 저장

In [69]:
import cv2
import numpy as np
import os
import noise
from opensimplex import OpenSimplex

# 노이즈 생성 함수들
def generate_perlin_noise(width, height, scale=10, intensity=0.5):
    perlin_noise = np.zeros((height, width))
    for i in range(height):
        for j in range(width):
            perlin_noise[i][j] = noise.pnoise2(i / scale, j / scale, octaves=6, persistence=0.5, lacunarity=2.0, repeatx=width, repeaty=height, base=0)
    perlin_noise = (perlin_noise - perlin_noise.min()) / (perlin_noise.max() - perlin_noise.min()) * 255 * intensity
    return perlin_noise.astype('uint8')

def generate_simplex_noise(width, height, scale=10, intensity=0.5):
    simplex = OpenSimplex(seed=1111)
    simplex_noise = np.zeros((height, width))
    for i in range(height):
        for j in range(width):
            simplex_noise[i][j] = simplex.noise2(i / scale, j / scale)
    simplex_noise = (simplex_noise - simplex_noise.min()) / (simplex_noise.max() - simplex_noise.min()) * 255 * intensity
    return simplex_noise.astype('uint8')

def generate_gaussian_noise(image, mean=0, var=0.1):
    sigma = var ** 0.5
    gauss = np.random.normal(mean, sigma, image.shape).astype('uint8')
    noisy_image = cv2.add(image, gauss)
    return noisy_image

def generate_poisson_noise(image):
    vals = len(np.unique(image))
    vals = 2 ** np.ceil(np.log2(vals))
    noisy = np.random.poisson(image * vals) / float(vals)
    return np.clip(noisy, 0, 255).astype('uint8')


# Inverted 효과
def apply_inverted(image):
    return cv2.bitwise_not(image)

# Saturated 효과
def apply_saturated(image, intensity=1.5):
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    hsv_image[:, :, 1] = cv2.multiply(hsv_image[:, :, 1], intensity)
    saturated_image = cv2.cvtColor(hsv_image, cv2.COLOR_HSV2BGR)
    return saturated_image

# Sepia 효과
def apply_sepia(image):
    kernel = np.array([[0.272, 0.534, 0.131],
                       [0.349, 0.686, 0.168],
                       [0.393, 0.769, 0.189]])
    sepia_image = cv2.transform(image, kernel)
    sepia_image = np.clip(sepia_image, 0, 255)
    return sepia_image

# Brightness 효과
def apply_brightness(image, brightness=30):
    bright_image = np.clip(image + brightness, 0, 255).astype(np.uint8)
    return bright_image

def apply_noise_to_image(image, noise_image, noise_weight=0.5):
    noise_image_resized = cv2.resize(noise_image, (image.shape[1], image.shape[0]))
    noisy_image = cv2.addWeighted(image, 1 - noise_weight, cv2.cvtColor(noise_image_resized, cv2.COLOR_GRAY2BGR), noise_weight, 0)
    return noisy_image

def save_images_with_noise(image, file_name, output_folder, noise_intensities):
    for intensity in noise_intensities:
        perlin_noise = generate_perlin_noise(image.shape[1], image.shape[0], intensity=intensity)
        simplex_noise = generate_simplex_noise(image.shape[1], image.shape[0], intensity=intensity)

        perlin_noise_image = apply_noise_to_image(image, perlin_noise, noise_weight=intensity)
        simplex_noise_image = apply_noise_to_image(image, simplex_noise, noise_weight=intensity)
        gaussian_noise_image = generate_gaussian_noise(image, var=intensity)
        poisson_noise_image = generate_poisson_noise(image)


        # 결과 저장
        cv2.imwrite(os.path.join(output_folder, f'{file_name}_perlin_noise_{intensity}.jpg'), perlin_noise_image)
        cv2.imwrite(os.path.join(output_folder, f'{file_name}_simplex_noise_{intensity}.jpg'), simplex_noise_image)
        cv2.imwrite(os.path.join(output_folder, f'{file_name}_gaussian_noise_{intensity}.jpg'), gaussian_noise_image)
        cv2.imwrite(os.path.join(output_folder, f'{file_name}_poisson_noise.jpg'), poisson_noise_image)

In [70]:
def apply_kernels(src_folder, output_folder, noise_intensities):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    for file in os.listdir(src_folder):
        file_name, file_ext = os.path.splitext(file)
        image_path = os.path.join(src_folder, file)
        image = cv2.imread(image_path)

        if image is None:
            continue

        inverted_image = apply_inverted(image)
        saturated_image = apply_saturated(image)
        sepia_image = apply_sepia(image)
        bright_image = apply_brightness(image)

        # 원본 및 효과 이미지를 저장
        cv2.imwrite(os.path.join(output_folder, f'{file_name}_original{file_ext}'), image)
        cv2.imwrite(os.path.join(output_folder, f'{file_name}_inverted{file_ext}'), inverted_image)
        cv2.imwrite(os.path.join(output_folder, f'{file_name}_saturated{file_ext}'), saturated_image)
        cv2.imwrite(os.path.join(output_folder, f'{file_name}_sepia{file_ext}'), sepia_image)
        cv2.imwrite(os.path.join(output_folder, f'{file_name}_bright{file_ext}'), bright_image)

        # 노이즈 추가 및 저장
        save_images_with_noise(image, f'{file_name}_original', output_folder, noise_intensities)
        save_images_with_noise(inverted_image, f'{file_name}_inverted', output_folder, noise_intensities)
        save_images_with_noise(saturated_image, f'{file_name}_saturated', output_folder, noise_intensities)
        save_images_with_noise(sepia_image, f'{file_name}_sepia', output_folder, noise_intensities)
        save_images_with_noise(bright_image, f'{file_name}_bright', output_folder, noise_intensities)

# 이미지 폴더 경로 설정
src_folder = 'C:/Users/Inbody/Desktop/OCR/data/background_settings/plain'
output_folder = 'C:/Users/Inbody/Desktop/OCR/data/background_settings/plain_add_noise'
noise_intensities = [0.05, 0.1, 0.15, 0.2, 0.25, 0.3]

apply_kernels(src_folder, output_folder, noise_intensities)
